In [1]:
import pandas as pd
import sys
sys.path.append("../../helper_functions")
import opstack_metadata_utils as ops
import duneapi_utils as d
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()


True

In [2]:
# Read the CSV file
df = pd.read_csv('chain_metadata_raw.csv')

In [3]:
# Trim columns
df.columns = df.columns.str.replace(" ", "").str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#Generate Alignment Column
df = ops.generate_alignment_column(df)

# Replace NaN values in object type columns with an empty string
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].fillna('')

# Save the cleaned DataFrame to a new CSV file
df.to_csv('../outputs/chain_metadata.csv', index=False)

In [4]:
# df

,chain_name,display_name,mainnet_chain_id,op_based_version,has_mods,is_law_of_chains,is_op_chain,raas_deployer,rpc_url,product_website,defillama_slug,l2beat_slug,growthepie_alias,gas_token,chain_layer,da_layer,is_in_superchain_registry_repo,alignment
0,op mainnet,OP Mainnet,10,bedrock,no,yes,yes,,https://mainnet.optimism.io,https://optimism.io,Optimism,optimism,OP Mainnet,ETH,L2,ethereum,yes,OP Chain
1,base,Base,8453,bedrock,no,yes,yes,,https://mainnet.base.org/,https://base.org,Base,base,Base,ETH,L2,ethereum,yes,OP Chain
2,zora,ZORA,7777777,bedrock,no,,yes,conduit,https://rpc.zora.energy/,https://zora.energy,Zora,zora,Zora,ETH,L2,ethereum,yes,OP Chain
3,public goods network,Public Goods Network,424,bedrock,yes,,,conduit,https://rpc.publicgoods.network/,https://publicgoods.network/,,publicgoodsnetwork,Public Goods Network,ETH,L2,ethereum,yes,OP Stack Fork
4,aevo network,Aevo,,legacy bedrock,yes,,,conduit,https://api.aevo.xyz,https://www.aevo.xyz/,protocols/aevo,aevo,,ETH,L2,ethereum,no,OP Stack Fork
5,opbnb,opBNB,204,bedrock,yes,,,,https://opbnb-mainnet-rpc.bnbchain.org,https://opbnb.bnbchain.org/en,opBNB,opbnb,,BNB,L2,bnb,no,OP Stack Fork
6,manta pacific,Manta Pacific,169,bedrock,yes,,,caldera,https://pacific-rpc.manta.network/http,https://pacific.manta.network/,Manta,mantapacific,,ETH,L2,celestia,no,OP Stack Fork
7,loot chain,Loot Chain,,bedrock,yes,,,caldera,https://rpc.lootchain.com/http,https://adventuregold.org/,,,,ETH,L2,polygon,no,OP Stack Fork
8,kroma network,Kroma,255,bedrock,yes,,,,https://api.kroma.network/,https://kroma.network/,Kroma,kroma,,ETH,L2,ethereum,no,OP Stack Fork
9,vienna,Vienna,,bedrock,yes,,,caldera,https://vienna.calderachain.xyz/http,https://vienna.caldera.dev/,,,,ETH,L2,unknown,no,OP Stack Fork


In [5]:
# Post to Dune API
d.write_dune_api_from_pandas(df, 'op_stack_chain_metadata_info_tracking',\
                             'Basic Info & Metadata about OP Stack Chains, including forks')

table at: dune.oplabspbc.dataset_op_stack_chain_metadata_info_tracking
Response status code: 200
Response content: b'{"success":true,"table_name":"op_stack_chain_metadata_info_tracking"}'
